Write your own GNN module

In [1]:
pip install dgl==0.4.3

Note: you may need to restart the kernel to use updated packages.


In [2]:
import dgl.data

c:\Users\MEKhani\anaconda3\envs\torch_env\lib\site-packages\dgl\base.py:25: UserWarning: Detected an old version of PyTorch. Suggest using torch>=1.2.0 for the best experience.
  warnings.warn(msg, warn_type)


In [3]:
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import dgl.function as fn
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
import torch.cuda


torch.__version__
torch.cuda.is_available()
print(dgl.__version__)

0.4.3


Message passing and GNNs

In [5]:
class SAGEConv(nn.Module):
    """Graph convolution module used by the GraphSAGE model.

    Parameters
    ----------
    in_feat : int
        Input feature size.
    out_feat : int
        Output feature size.
    """

    def __init__(self, in_feat, out_feat):
        super(SAGEConv, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        """Forward computation

        Parameters
        ----------
        g : Graph
            The input graph.
        h : Tensor
            The input node feature.
        """
        with g.local_scope():
            g.ndata["h"] = h
            # update_all is a message passing API.
            g.update_all(
                message_func=fn.copy_u("h", "m"),
                reduce_func=fn.mean("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

In [40]:
class Model(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(Model, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats)
        self.conv2 = SAGEConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

In [41]:
from dgl.data import CoraDataset

dataset = CoraDataset()
g = dataset[0]
# print(g)
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    all_logits = []
    best_val_acc = 0
    best_test_acc = 0
    features = g.ndata["feat"]
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    val_mask = g.ndata["val_mask"]
    test_mask = g.ndata["test_mask"]
    print(labels)
    
    for e in range(200):
        # Forward
        logits = model(g, features)
        # Compute prediction
        pred = logits.argmax(1)
      
        # Compute loss
        # Note that we should only compute the losses of the nodes in the training set,
        # i.e. with train_mask 1.
        train_mask = train_mask.byte()  
        val_mask = val_mask.byte()  
        test_mask = test_mask.byte()  
        labels = labels.long()
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])
        
        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()
        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        all_logits.append(logits.detach())
        if e % 5 == 0:
            print(
                "In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})".format(
                    e, loss, val_acc, best_val_acc, test_acc, best_test_acc
                )
            )
model = Model(g.ndata["feat"].shape[1], 16, 7)#dataset.num_classes)
train(g, model)

tensor([2, 5, 4,  ..., 1, 0, 2])
In epoch 0, loss: 1.951, val acc: 0.083 (best 0.083), test acc: 0.159 (best 0.159)
In epoch 5, loss: 1.852, val acc: 0.360 (best 0.360), test acc: 0.232 (best 0.232)
In epoch 10, loss: 1.672, val acc: 0.500 (best 0.567), test acc: 0.391 (best 0.431)
In epoch 15, loss: 1.447, val acc: 0.440 (best 0.567), test acc: 0.376 (best 0.431)
In epoch 20, loss: 1.202, val acc: 0.473 (best 0.567), test acc: 0.381 (best 0.431)
In epoch 25, loss: 0.939, val acc: 0.630 (best 0.630), test acc: 0.475 (best 0.475)
In epoch 30, loss: 0.705, val acc: 0.713 (best 0.713), test acc: 0.618 (best 0.618)
In epoch 35, loss: 0.509, val acc: 0.730 (best 0.730), test acc: 0.638 (best 0.638)
In epoch 40, loss: 0.355, val acc: 0.730 (best 0.730), test acc: 0.648 (best 0.638)
In epoch 45, loss: 0.239, val acc: 0.753 (best 0.753), test acc: 0.680 (best 0.680)
In epoch 50, loss: 0.156, val acc: 0.797 (best 0.797), test acc: 0.738 (best 0.738)
In epoch 55, loss: 0.102, val acc: 0.797 (bes

In [22]:
class WeightedSAGEConv(nn.Module):
    """Graph convolution module used by the GraphSAGE model with edge weights.

    Parameters
    ----------
    in_feat : int
        Input feature size.
    out_feat : int
        Output feature size.
    """

    def __init__(self, in_feat, out_feat):
        super(WeightedSAGEConv, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h, w):
        """Forward computation

        Parameters
        ----------
        g : Graph
            The input graph.
        h : Tensor
            The input node feature.
        w : Tensor
            The edge weight.
        """
        with g.local_scope():
            g.ndata["h"] = h
            g.edata["w"] = w
            g.update_all(
                message_func=fn.u_mul_e("h", "w", "m"),
                reduce_func=fn.mean("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

In [43]:
class Model(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(Model, self).__init__()
        self.conv1 = WeightedSAGEConv(in_feats, h_feats)
        self.conv2 = WeightedSAGEConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        # print(g.number_of_edges())
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(device)
        edge_weights = torch.ones(g.number_of_edges(), 1).to(device)
        h = self.conv1(g, in_feat, edge_weights)
        h = F.relu(h)
        h = self.conv2(g, h, torch.ones(edge_weights, 1).to(device))
        return h


model = Model(g.ndata["feat"].shape[1], 16, dataset.num_labels)

train(g, model)

tensor([2, 5, 4,  ..., 1, 0, 2])
cuda


DGLError: [07:09:53] C:\Users\Administrator\dgl\src\kernel\binary_reduce.cc:186: Check failed: ctx == arrays[i]->ctx (CPU:0 vs. GPU:0) : Expected device context CPU:0. But got GPU:0 for rhs_data.